In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from my_models.inference import get_rnn, get_cnn

In [2]:
TRAIN = pd.read_csv("TRAIN.csv")
DEV = pd.read_csv("DEV.csv")

aspect_order = tuple(DEV.aspectCategory.unique())
print(aspect_order)

df_aspect = TRAIN[['id','text', 'aspectCategory']]
df_sentiment = TRAIN[['id','text', 'polarity']]

('price', 'food', 'ambience', 'service', 'anecdotes/miscellaneous')


# Read embedding

In [72]:
from my_models import utils

emb_dim = 100
vocab, embedding_matrix = utils.get_embeddings(emb_dim)

maxlen = 30
vocab_size = embedding_matrix.shape[0]

In [4]:
def drop_dups(df):
    if any(df.duplicated()):
        dups = df[df.duplicated()]
        return df.drop_duplicates()
    
df_aspect = drop_dups(df_aspect)
df_sentiment = drop_dups(df_sentiment)

# 0) Baseline

# 1) Sentiment

In [5]:
from my_models import sentiment

X_TRAIN_sent, X_DEV_sent, Y_TRAIN_sent, Y_DEV_sent = TRAIN['text'], DEV['text'], TRAIN['polarity'], DEV['polarity']

## 1.1) Log reg - bow

In [6]:
def gym_sentiment_ml(
    feature_mode, 
    model, 
    X_TRAIN_sent=X_TRAIN_sent, X_DEV_sent=X_DEV_sent, 
    Y_TRAIN_sent=Y_TRAIN_sent, Y_DEV_sent=Y_DEV_sent, 
    **kwargs
):
    sent_model = sentiment.ml(feature_mode, model, **kwargs)
    
    # preprocess
    X_train_sent = sent_model.preprocess(X_TRAIN_sent.values)
    X_dev_sent   = sent_model.preprocess(X_DEV_sent.values)

    # train
    sent_model.fit(X_train_sent, Y_TRAIN_sent)

    # inference
    y_pred = sent_model.predict(X_dev_sent)
    utils.get_reports(y_true = Y_DEV_sent, y_pred=y_pred)
    return sent_model

In [7]:
from sklearn.linear_model import LogisticRegression

logreg_sent_bow = gym_sentiment_ml("BOW", LogisticRegression, max_iter=200)

Creating new BOW vectorizer...
BOW matrix: (2365, 3570)
              precision    recall  f1-score   support

    conflict       0.29      0.13      0.18        47
    negative       0.59      0.48      0.53       178
     neutral       0.45      0.30      0.36       100
    positive       0.75      0.89      0.81       464

    accuracy                           0.68       789
   macro avg       0.52      0.45      0.47       789
weighted avg       0.65      0.68      0.65       789



## 1.2) Log reg - tfidf

In [8]:
logreg_sent_bow = gym_sentiment_ml("TFIDF", LogisticRegression, max_iter=200)

Creating new TFIDF vectorizer...
TFIDF matrix: (2365, 3570)
              precision    recall  f1-score   support

    conflict       1.00      0.02      0.04        47
    negative       0.65      0.45      0.53       178
     neutral       0.76      0.16      0.26       100
    positive       0.69      0.96      0.81       464

    accuracy                           0.69       789
   macro avg       0.78      0.40      0.41       789
weighted avg       0.71      0.69      0.63       789



## 1.3) bowNN

In [9]:
def gym_sentiment_dl(
    is_bow,
    model_template,
    compile_info,
    epochs = 5,
    use_pretrained = True,
    X_TRAIN_sent=X_TRAIN_sent, X_DEV_sent=X_DEV_sent, 
    Y_TRAIN_sent=Y_TRAIN_sent, Y_DEV_sent=Y_DEV_sent, 
    **kwargs
):
    if use_pretrained:
        print("use pretrained emb")
        model = sentiment.dl_pretrained(vocab, compile_info)
    else:
        model = sentiment.dl(compile_info, is_bow)
    model.set_model_template(model_template)
    
    X_train_sent,Y_train_sent = model.preprocess(X_TRAIN_sent.values, Y_TRAIN_sent.values, **kwargs)
    X_dev_sent,Y_dev_sent     = model.preprocess(X_DEV_sent.values, Y_DEV_sent.values, **kwargs)
    
    history = model.fit(
                    X_train_sent, Y_train_sent, X_dev_sent, Y_dev_sent,
                    batch_size = 32, epochs = epochs
                )
    
    y_pred = model.predict(X_dev_sent)
    utils.get_reports(
        y_true = [model.le.classes_[i] for i in Y_dev_sent], 
        y_pred= [model.le.classes_[i] for i in y_pred]
    )
    return model, history, (X_train_sent, Y_train_sent), (X_dev_sent, Y_dev_sent)

In [10]:
import tensorflow as tf

def get_bowNN(dense_layers):
    model = tf.keras.models.Sequential()
    for units in dense_layers:
        model.add(tf.keras.layers.Dense(units, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(len(Y_TRAIN_sent.unique()), activation='softmax'))
    
    return model

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
compile_info = dict(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [11]:
bow_nn, bow_nn_hist, train_data_sent, dev_data_sent = gym_sentiment_dl(True, get_bowNN([1000,256]), compile_info, use_pretrained=False)

...Adapting new Tokenizer
...Build new LabelEncoder
Epoch 1/5
74/74 [==============================] - 2s 22ms/step - loss: 1.0386 - accuracy: 0.5911 - val_loss: 0.9132 - val_accuracy: 0.6274
Epoch 2/5
74/74 [==============================] - 1s 20ms/step - loss: 0.6927 - accuracy: 0.7230 - val_loss: 0.8250 - val_accuracy: 0.7022
Epoch 3/5
74/74 [==============================] - 2s 22ms/step - loss: 0.4271 - accuracy: 0.8465 - val_loss: 0.8918 - val_accuracy: 0.6895
Epoch 4/5
74/74 [==============================] - 2s 20ms/step - loss: 0.2822 - accuracy: 0.9146 - val_loss: 0.9602 - val_accuracy: 0.6958
Epoch 5/5
74/74 [==============================] - 2s 21ms/step - loss: 0.2182 - accuracy: 0.9366 - val_loss: 1.0209 - val_accuracy: 0.7009
              precision    recall  f1-score   support

    conflict       0.41      0.23      0.30        47
    negative       0.63      0.52      0.57       178
     neutral       0.44      0.34      0.38       100
    positive       0.77      0.

### 1.3.1) tfidf NN

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
compile_info = dict(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics='accuracy')

tfidf_nn, tfidf_nn_hist, train_data, dev_data = gym_sentiment_dl('tfidf', get_bowNN([1000,256]), compile_info, use_pretrained=False)

...Adapting new Tokenizer
...Build new LabelEncoder
Epoch 1/5
74/74 [==============================] - 2s 22ms/step - loss: 1.0546 - accuracy: 0.5839 - val_loss: 0.8906 - val_accuracy: 0.6667
Epoch 2/5
74/74 [==============================] - 2s 22ms/step - loss: 0.5972 - accuracy: 0.7831 - val_loss: 0.9428 - val_accuracy: 0.7034
Epoch 3/5
74/74 [==============================] - 2s 24ms/step - loss: 0.3733 - accuracy: 0.8875 - val_loss: 1.0359 - val_accuracy: 0.6984
Epoch 4/5
74/74 [==============================] - 2s 26ms/step - loss: 0.2627 - accuracy: 0.9260 - val_loss: 1.1312 - val_accuracy: 0.7047
Epoch 5/5
74/74 [==============================] - 2s 32ms/step - loss: 0.2248 - accuracy: 0.9442 - val_loss: 1.2688 - val_accuracy: 0.6907
              precision    recall  f1-score   support

    conflict       0.41      0.15      0.22        47
    negative       0.68      0.49      0.57       178
     neutral       0.42      0.30      0.35       100
    positive       0.74      0.

## 1.4) Bidir-GRU (GloVe)

In [16]:
vocab_size = maxtokens = 5000

rnn_params = dict(
    rnn_layers=[128], 
    dense_layers=[64], 
    embedding_matrix=embedding_matrix, 
    n_outputs=len(Y_TRAIN_sent.unique()), 
    embedding_trainable=True,
    #vocab_size = vocab_size,
    #emb_dim = 256
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
compile_info = dict(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics='accuracy')

bidir_sent, bidir_sent_hist, train_data_sent, dev_data_sent = gym_sentiment_dl(False, get_rnn(**rnn_params), compile_info)

Using pretrained word embedding
use pretrained emb
...Build new LabelEncoder
Epoch 1/5
74/74 [==============================] - 58s 611ms/step - loss: 1.3717 - accuracy: 0.5268 - val_loss: 1.0726 - val_accuracy: 0.5881
Epoch 2/5
74/74 [==============================] - 41s 556ms/step - loss: 0.9943 - accuracy: 0.6034 - val_loss: 0.9919 - val_accuracy: 0.5995
Epoch 3/5
74/74 [==============================] - 42s 561ms/step - loss: 0.8486 - accuracy: 0.6486 - val_loss: 1.0533 - val_accuracy: 0.5488
Epoch 4/5
74/74 [==============================] - 41s 549ms/step - loss: 0.7261 - accuracy: 0.7125 - val_loss: 1.4412 - val_accuracy: 0.6451
Epoch 5/5
74/74 [==============================] - 42s 562ms/step - loss: 0.6788 - accuracy: 0.7408 - val_loss: 1.0358 - val_accuracy: 0.6286
              precision    recall  f1-score   support

    conflict       0.50      0.04      0.08        47
    negative       0.50      0.46      0.48       178
     neutral       0.29      0.39      0.33       

## 1.5) CNN (GloVe)

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, GlobalAveragePooling1D
def make_cnn_model(n_outputs, embedding_trainable=False):
    filters = 250 
    kernel_size = 3 
    hidden_dims = 250 
    vocab_size, emb_dim = embedding_matrix.shape

    cnn_model = Sequential()
    cnn_model.add(
        Embedding(
                input_dim=vocab_size,
                output_dim=emb_dim, 
                embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                mask_zero=True,
                trainable=embedding_trainable
                ) 
    )
    
    cnn_model.add(Conv1D(filters,
                        kernel_size,
                        activation='relu',
                        strides=1))

    cnn_model.add(GlobalAveragePooling1D())
    cnn_model.add(Dropout(0.5))

    cnn_model.add(Dense(hidden_dims, activation='relu'))
    cnn_model.add(Dropout(0.5))
    cnn_model.add(Dense(n_outputs, activation='softmax'))

    return cnn_model

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
compile_info = dict(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics='accuracy')

n_outputs = len(Y_TRAIN_sent.unique())
cnn_sent, cn_sent_hist, train_data_sent, dev_data_sent = gym_sentiment_dl(False, make_cnn_model(n_outputs, embedding_trainable=True), compile_info, epochs=5)

use pretrained emb
...Build new LabelEncoder
Epoch 1/5
74/74 [==============================] - 41s 540ms/step - loss: 1.0933 - accuracy: 0.5793 - val_loss: 1.0156 - val_accuracy: 0.5906
Epoch 2/5
74/74 [==============================] - 36s 486ms/step - loss: 0.8841 - accuracy: 0.6592 - val_loss: 0.8708 - val_accuracy: 0.6679
Epoch 3/5
74/74 [==============================] - 37s 504ms/step - loss: 0.7103 - accuracy: 0.7175 - val_loss: 0.9649 - val_accuracy: 0.6717
Epoch 4/5
74/74 [==============================] - 37s 497ms/step - loss: 0.5977 - accuracy: 0.7683 - val_loss: 1.0543 - val_accuracy: 0.6692
Epoch 5/5
74/74 [==============================] - 37s 494ms/step - loss: 0.4891 - accuracy: 0.8131 - val_loss: 0.9926 - val_accuracy: 0.6616
              precision    recall  f1-score   support

    conflict       0.57      0.09      0.15        47
    negative       0.57      0.49      0.53       178
     neutral       0.34      0.43      0.38       100
    positive       0.77     

# 2) Aspects

In [73]:
def swapCol(df):    
    df = pd.DataFrame(
        {aspect: df[aspect] for aspect in aspect_order}
    )
    assert aspect_order == tuple(df.columns)
    return df

def prep_aspect_df(df_aspect):
    temp_df = pd.pivot_table(
                    df_aspect,
                    index='text',
                    values='aspectCategory',
                    aggfunc=lambda x: list(x)
                )

    for a in aspect_order:
        temp_df[a] = temp_df.apply(lambda x: 1 if a in x.aspectCategory else 0, axis=1)

    temp_df.drop(['aspectCategory'], axis=1, inplace=True)
    print(temp_df.columns)
    return temp_df

TRAIN_aspect = prep_aspect_df(TRAIN)
DEV_aspect = prep_aspect_df(DEV)

Index(['price', 'food', 'ambience', 'service', 'anecdotes/miscellaneous'], dtype='object')
Index(['price', 'food', 'ambience', 'service', 'anecdotes/miscellaneous'], dtype='object')


In [74]:
X_TRAIN_asp, X_DEV_asp, Y_TRAIN_asp, Y_DEV_asp = TRAIN_aspect.index, DEV_aspect.index, TRAIN_aspect, DEV_aspect
all(Y_DEV_asp.columns == Y_TRAIN_asp.columns)

True

## 2.1) Logreg - bow

In [15]:
from my_models import aspect

def gym_aspect_ml(
    feature_mode, model,
    X_TRAIN_sent=X_TRAIN_asp, X_DEV_sent=X_DEV_asp, 
    Y_TRAIN_sent=Y_TRAIN_asp, Y_DEV_sent=Y_DEV_asp, 
    **kwargs
):
    model = aspect.ml(feature_mode, model)
    
    X_train_asp = model.preprocess(X_TRAIN_asp, Y_TRAIN_asp)
    X_dev_asp = model.preprocess(X_DEV_asp, Y_DEV_asp)
        
    model.fit(X_train_asp, Y_TRAIN_asp)
    
    assert all(Y_TRAIN_asp.columns == Y_DEV_asp.columns)
    assert all(model.classes == Y_DEV_asp.columns)
    
    outputs, outputs_prob = model.predict(X_dev_asp)    
    utils.get_reports(
        y_true = Y_DEV_asp.reset_index(drop=True), 
        y_pred= outputs
    )
    return model, (X_train_asp, Y_TRAIN_asp), (X_dev_asp, Y_DEV_sent)
    

In [16]:
from sklearn.linear_model import LogisticRegression

logreg_asp_bow, train_data_asp, dev_data_asp = gym_aspect_ml("BOW", model=LogisticRegression)

Creating new BOW vectorizer...
BOW matrix: (2034, 3570)
Creating new models
predicting price...
predicting food...
predicting ambience...
predicting service...
predicting anecdotes/miscellaneous...
              precision    recall  f1-score   support

           0       0.31      0.27      0.29        71
           1       0.62      0.72      0.67       261
           2       0.37      0.26      0.31        96
           3       0.55      0.56      0.55       134
           4       0.66      0.85      0.74       227

   micro avg       0.58      0.64      0.61       789
   macro avg       0.50      0.53      0.51       789
weighted avg       0.56      0.64      0.59       789
 samples avg       0.61      0.64      0.62       789



## 2.2) Logreg - tfidf

In [41]:
logreg_asp_tfidf, train_data_asp, dev_data_asp = gym_aspect_ml("TFIDF", model=LogisticRegression)

Creating new TFIDF vectorizer...
TFIDF matrix: (2034, 3570)
Creating new models
predicting price...
predicting food...
predicting ambience...
predicting service...
predicting anecdotes/miscellaneous...
              precision    recall  f1-score   support

           0       0.50      0.10      0.16        71
           1       0.62      0.83      0.71       261
           2       0.54      0.16      0.24        96
           3       0.60      0.43      0.50       134
           4       0.64      0.85      0.73       227

   micro avg       0.62      0.62      0.62       789
   macro avg       0.58      0.47      0.47       789
weighted avg       0.60      0.62      0.57       789
 samples avg       0.63      0.63      0.62       789



## 2.3) Bidir-GRU (GloVe) [multiple binary]

In [85]:
def gym_aspect_dl(
    is_bow,
    model_template,
    compile_info,
    epochs = 5,
    n_models = 5,  # 5 aspects in total
    use_pretrained = True,
    X_TRAIN_sent=X_TRAIN_asp, X_DEV_sent=X_DEV_asp, 
    Y_TRAIN_sent=Y_TRAIN_asp, Y_DEV_sent=Y_DEV_asp, 
    **kwargs
):
    
    if use_pretrained:
        model = aspect.dl_pretrained(vocab, compile_info, n_models)
    else:
        model = aspect.dl(compile_info, n_models, is_bow)
    
    model.set_model_template(model_template)
    
    X_train_asp,Y_train_asp = model.preprocess(X_TRAIN_asp, Y_TRAIN_asp, **kwargs)
    X_dev_asp,Y_dev_asp     = model.preprocess(X_DEV_asp, Y_DEV_asp, **kwargs)

    
    history = model.fit(
                    X_train_asp, Y_train_asp, X_dev_asp, Y_dev_asp,
                    batch_size = 32, epochs = epochs
                )
    assert all(Y_train_asp.columns == Y_dev_asp.columns)
    assert all(model.classes == Y_dev_asp.columns)
    
    outputs, outputs_prob = model.predict(X_dev_asp)        
    utils.get_reports(
        y_true = Y_DEV_asp.reset_index(drop=True), 
        y_pred= outputs
    )
    return model, history, (X_train_asp, Y_train_asp), (X_dev_asp, Y_dev_asp)

In [76]:
rnn_params = dict(
    rnn_layers=[128], 
    dense_layers=[64], 
    embedding_matrix=embedding_matrix, 
    n_outputs=1, 
    embedding_trainable=True,
    #vocab_size = vocab_size,
    #emb_dim = 256
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
compile_info = dict(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

bidir_asp, bidir_asp_hist, train_data_asp, dev_data = gym_aspect_dl(False, get_rnn(**rnn_params), compile_info, epochs=3)

Using pretrained word embedding
fitting price ...

Epoch 1/3
64/64 [==============================] - 43s 550ms/step - loss: 0.5173 - accuracy: 0.7714 - val_loss: 0.3359 - val_accuracy: 0.9055
Epoch 2/3
64/64 [==============================] - 31s 490ms/step - loss: 0.3824 - accuracy: 0.8776 - val_loss: 0.3312 - val_accuracy: 0.9055
Epoch 3/3
64/64 [==============================] - 33s 508ms/step - loss: 0.3613 - accuracy: 0.8791 - val_loss: 0.3305 - val_accuracy: 0.9015
fitting food ...

Epoch 1/3
64/64 [==============================] - 52s 624ms/step - loss: 0.8263 - accuracy: 0.5757 - val_loss: 0.6366 - val_accuracy: 0.6551
Epoch 2/3
64/64 [==============================] - 35s 553ms/step - loss: 0.6113 - accuracy: 0.6962 - val_loss: 0.5828 - val_accuracy: 0.7403
Epoch 3/3
64/64 [==============================] - 39s 618ms/step - loss: 0.5268 - accuracy: 0.7384 - val_loss: 0.5399 - val_accuracy: 0.7204
fitting ambience ...

Epoch 1/3
64/64 [==============================] - 61s 72

## 2.4) CNN (GloVe) [Multiple binary]

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, GlobalAveragePooling1D

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
compile_info = dict(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

def make_cnnAsp_model(embedding_trainable=False):
    filters = 128 
    kernel_size = 5 
    hidden_dims = 250 
    vocab_size, emb_dim = embedding_matrix.shape

    cnn_model = Sequential()
    cnn_model.add(
        Embedding(
                input_dim=vocab_size,
                output_dim=emb_dim, 
                embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                mask_zero=True,
                trainable=embedding_trainable
                ) 
    )
    
    cnn_model.add(Conv1D(filters,
                        kernel_size,
                        activation='relu',
                        strides=2))

    cnn_model.add(GlobalAveragePooling1D())
    cnn_model.add(Dropout(0.5))

    cnn_model.add(Dense(hidden_dims, activation='relu'))
    cnn_model.add(Dropout(0.5))
    cnn_model.add(Dense(1, activation='sigmoid'))

    return cnn_model

cnn_asp, cnn_asp_hist, train_data_asp, dev_data_asp = gym_aspect_dl(False, make_cnnAsp_model(embedding_trainable=True), compile_info, epochs=3)

fitting price ...

Epoch 1/3
64/64 [==============================] - 16s 240ms/step - loss: 0.3824 - accuracy: 0.8899 - val_loss: 0.3206 - val_accuracy: 0.9055
Epoch 2/3
64/64 [==============================] - 14s 224ms/step - loss: 0.3185 - accuracy: 0.8997 - val_loss: 0.3124 - val_accuracy: 0.9055
Epoch 3/3
64/64 [==============================] - 14s 224ms/step - loss: 0.2016 - accuracy: 0.9154 - val_loss: 0.3951 - val_accuracy: 0.8895
fitting food ...

Epoch 1/3
64/64 [==============================] - 17s 244ms/step - loss: 0.6013 - accuracy: 0.6996 - val_loss: 0.6919 - val_accuracy: 0.6671
Epoch 2/3
64/64 [==============================] - 15s 231ms/step - loss: 0.3425 - accuracy: 0.8746 - val_loss: 0.5548 - val_accuracy: 0.7510
Epoch 3/3
64/64 [==============================] - 14s 224ms/step - loss: 0.1948 - accuracy: 0.9371 - val_loss: 0.8611 - val_accuracy: 0.7230
fitting ambience ...

Epoch 1/3
64/64 [==============================] - 17s 245ms/step - loss: 0.4599 - accura

## 2.5) bow NN [multi binary]

In [97]:
def get_NN_asp(dense_layers):
    model = tf.keras.models.Sequential()
    for units in dense_layers:
        model.add(tf.keras.layers.Dense(units, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
compile_info = dict(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')
nn_bow_asp, nn_bow_asp_hist, train_data_asp, dev_data_asp = gym_aspect_dl('tfidf', get_NN_asp([1200, 200]), compile_info, epochs=1, use_pretrained=False)

...Adapting new Tokenizer
fitting price ...

64/64 [==============================] - 2s 29ms/step - loss: 0.5035 - accuracy: 0.8874 - val_loss: 0.3372 - val_accuracy: 0.8948
fitting food ...

64/64 [==============================] - 2s 28ms/step - loss: 0.6667 - accuracy: 0.7183 - val_loss: 0.7205 - val_accuracy: 0.7710
fitting ambience ...

64/64 [==============================] - 3s 31ms/step - loss: 0.6211 - accuracy: 0.8255 - val_loss: 0.4292 - val_accuracy: 0.8695
fitting service ...

64/64 [==============================] - 3s 32ms/step - loss: 0.7557 - accuracy: 0.8053 - val_loss: 0.6323 - val_accuracy: 0.8469
fitting anecdotes/miscellaneous ...

64/64 [==============================] - 3s 38ms/step - loss: 0.7868 - accuracy: 0.7301 - val_loss: 0.4440 - val_accuracy: 0.7776
predicting price...
predicting food...
predicting ambience...
predicting service...
predicting anecdotes/miscellaneous...
              precision    recall  f1-score   support

           0       0.27      0

## 2.5) GRU (GloVe) [1 multi label]

In [77]:
def gym_aspect_mulabel(
    is_bow,
    model_template,
    compile_info,
    epochs = 5,
    use_pretrained = True,
    X_TRAIN_asp=X_TRAIN_asp, X_DEV_asp=X_DEV_asp, 
    Y_TRAIN_asp=Y_TRAIN_asp, Y_DEV_asp=Y_DEV_asp, 
    **kwargs
):
    
    if use_pretrained:
        model = aspect.mulabel_pretrained(vocab, compile_info, is_bow=None, le=None)
    else:
        model = aspect.mulabel(compile_info, is_bow)
    
    model.set_model_template(model_template)
    
    X_train_asp,Y_train_asp = model.preprocess(X_TRAIN_asp, Y_TRAIN_asp, **kwargs)
    X_dev_asp,Y_dev_asp     = model.preprocess(X_DEV_asp, Y_DEV_asp, **kwargs)
    
    history = model.fit(
                    X_train_asp, Y_train_asp, X_dev_asp, Y_dev_asp,
                    batch_size = 32, epochs = epochs
                )
    
    outputs, outputs_prob = model.predict(X_dev_asp)        
    utils.get_reports(
        y_true = Y_dev_asp.reset_index(drop=True), 
        y_pred= outputs
    )
    return model, history, (X_train_asp, Y_train_asp), (X_dev_asp, Y_dev_asp)

In [78]:
from my_models.inference import BaseModel
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout, Conv1D, GlobalMaxPooling1D, BatchNormalization

def SigmoidEntropy(y_true, y_pred):
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=tf.cast(y_true,tf.float32))
    return tf.reduce_mean(tf.reduce_sum(cross_entropy, axis=1))
    
def get_gru_mulabel_template(rnn_layers, dense_layers, embedding_matrix, n_outputs, embedding_trainable=False):
    model = BaseModel(embedding_matrix, embedding_trainable=embedding_trainable)
    
    for rnn_unit in rnn_layers[:-1]:
        model.add( Bidirectional(GRU(rnn_unit, dropout=0.5, return_sequences=True)) )
    model.add( Bidirectional(GRU(rnn_layers[-1], dropout=0.5, return_sequences=False)) )

    model.add(BatchNormalization())    
    
    for dense_unit in dense_layers:
        model.add( Dense(dense_unit, activation='relu') )
        model.add(Dropout(0.5))
    model.add( Dense(n_outputs) )  
      
    return model

In [80]:
embedding_matrix.shape

(400001, 100)

In [79]:
gru_malabel_params = dict(
    rnn_layers=[200], 
    dense_layers=[32], 
    embedding_matrix=embedding_matrix, 
    n_outputs=len(Y_TRAIN_asp.columns), 
    embedding_trainable=True
    #vocab_size = vocab_size,
    #emb_dim = 256
)
compile_info = dict(optimizer='adam', loss=SigmoidEntropy, metrics='accuracy')

gru_mulabel_template = get_gru_mulabel_template(**gru_malabel_params)
gru_mulabel, gru_mulabel_hist, train_data_asp, dev_data_asp = gym_aspect_mulabel(False, gru_mulabel_template, compile_info, epochs=10)

Using pretrained word embedding
model is compiled
Epoch 1/10
64/64 [==============================] - 46s 608ms/step - loss: 3.5336 - accuracy: 0.2507 - val_loss: 2.7541 - val_accuracy: 0.4341
Epoch 2/10
64/64 [==============================] - 36s 567ms/step - loss: 2.8722 - accuracy: 0.3702 - val_loss: 2.6182 - val_accuracy: 0.4621
Epoch 3/10
64/64 [==============================] - 36s 562ms/step - loss: 2.5736 - accuracy: 0.4385 - val_loss: 2.4064 - val_accuracy: 0.4794
Epoch 4/10
64/64 [==============================] - 37s 579ms/step - loss: 2.3894 - accuracy: 0.4946 - val_loss: 2.3193 - val_accuracy: 0.4967
Epoch 5/10
64/64 [==============================] - 36s 556ms/step - loss: 2.3035 - accuracy: 0.5088 - val_loss: 2.2026 - val_accuracy: 0.5033
Epoch 6/10
64/64 [==============================] - 38s 595ms/step - loss: 2.1654 - accuracy: 0.5506 - val_loss: 2.2641 - val_accuracy: 0.5060
Epoch 7/10
64/64 [==============================] - 38s 589ms/step - loss: 2.0158 - accuracy

## 2.6) Bow NN (BOW) [1 multi label]

In [33]:
import tensorflow as tf

def get_bowNN_asp_mulabel(dense_layers):
    model = tf.keras.models.Sequential()
    for units in dense_layers:
        model.add(tf.keras.layers.Dense(units, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(len(Y_TRAIN_asp.columns.unique())))
    
    return model

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
compile_info = dict(optimizer=optimizer, loss=SigmoidEntropy, metrics='accuracy')

In [40]:
get_bowNN_asp_mulabel_params = dict(dense_layers=[1000, 258])

bowNN_asp_mulabel_template = get_bowNN_asp_mulabel(**get_bowNN_asp_mulabel_params)
bowNN_asp_mulabel, bowNN_asp_mulabel_hist, train_data_asp, dev_data_asp = gym_aspect_mulabel('tfidf', bowNN_asp_mulabel_template, compile_info, epochs=3, use_pretrained=False)

model is compiled
...Adapting new Tokenizer
Epoch 1/3
64/64 [==============================] - 2s 24ms/step - loss: 2.4588 - accuracy: 0.4853 - val_loss: 2.0699 - val_accuracy: 0.5619
Epoch 2/3
64/64 [==============================] - 1s 20ms/step - loss: 1.3301 - accuracy: 0.7404 - val_loss: 2.4337 - val_accuracy: 0.5846
Epoch 3/3
64/64 [==============================] - 1s 21ms/step - loss: 0.6387 - accuracy: 0.8579 - val_loss: 3.5043 - val_accuracy: 0.5526
              precision    recall  f1-score   support

           0       0.27      0.20      0.23        71
           1       0.58      0.68      0.63       261
           2       0.37      0.27      0.31        96
           3       0.46      0.43      0.44       134
           4       0.64      0.78      0.70       227

   micro avg       0.55      0.57      0.56       789
   macro avg       0.47      0.47      0.46       789
weighted avg       0.53      0.57      0.54       789
 samples avg       0.57      0.58      0.57     

# 3) Inference

In [19]:
logreg_asp_bow.classes

Index(['food', 'price', 'ambience', 'service', 'anecdotes/miscellaneous'], dtype='object')

In [65]:
from my_models import inference

inferencer = inference.InferenceModel(tfidf_nn, logreg_asp_tfidf)

df_train = pd.read_csv("contest1_test.csv")

df_train_inference = df_train[['id','text']]
outputs = inferencer.predict(df_train_inference)

predicting food...
predicting price...
predicting ambience...
predicting service...
predicting anecdotes/miscellaneous...


In [66]:
## if use DL as a sent model, run this
outputs['polarity'] = outputs['polarity'].apply(lambda x:tfidf_nn.le.inverse_transform([x])[0])

In [67]:
outputs.to_csv("resulting_predictions/test_pred/tfidf_nn_logreg_asp_bow.csv")

In [68]:
outputs

,aspectCategory,polarity
id,,
899,food,neutral
899,service,neutral
1349,anecdotes/miscellaneous,positive
934,food,positive
2199,food,positive
...,...,...
1063,anecdotes/miscellaneous,positive
777,food,positive
875,anecdotes/miscellaneous,neutral


In [64]:
TEST = pd.read_csv("DEV.csv")
TEST.head()

,id,text,aspectCategory,polarity
0,1315,Amma has the worst value for money I have expe...,price,negative
1,2576,By far the best salad I have had in a fast foo...,food,positive
2,2850,"The food was amazing, the service was so atten...",ambience,positive
3,301,"There was a long wait for a table outside, but...",service,conflict
4,87,"Having hunted around for a quiet, romantic, ye...",anecdotes/miscellaneous,neutral


In [65]:
logreg_asp_tfidf_input = logreg_asp_tfidf.preprocess(TEST.text)
pred, pred_prob = logreg_asp_tfidf.predict(logreg_asp_tfidf_input)

predicting price...
predicting food...
predicting ambience...
predicting service...
predicting anecdotes/miscellaneous...


In [66]:
output_aspects = []
for row in pred.values:
    temp = []
    for i, p in enumerate(row):
        if p > 0:
            temp.append(logreg_asp_tfidf.classes[i])
    output_aspects.append(temp)

In [67]:
pred_sent = pd.read_csv("dev_pred_sentiment.csv")
pred_sent

,id,polarity
0,1315,negative
1,2576,positive
2,2850,positive
3,301,positive
4,87,neutral
...,...,...
784,1082,positive
785,3243,neutral
786,1191,negative
787,1380,positive


In [68]:
pred_df = pd.DataFrame({
    'id':pred_sent.id,
    'aspectCategory': output_aspects,
    'polarity': pred_sent.polarity
})

In [69]:
pred_df.explode('aspectCategory')

,id,aspectCategory,polarity
0,1315,anecdotes/miscellaneous,negative
1,2576,food,positive
2,2850,food,positive
2,2850,service,positive
3,301,service,positive
...,...,...,...
784,1082,food,positive
785,3243,anecdotes/miscellaneous,neutral
786,1191,food,negative
787,1380,anecdotes/miscellaneous,positive


In [70]:
pred_df.explode('aspectCategory').to_csv("resulting_predictions/dev/bert_logreg_asp_tfidf.csv", index=False)

In [71]:
!python evaluate.py DEV.csv resulting_predictions/dev/bert_logreg_asp_tfidf.csv

=== CLASSIFICATION : ASPECT ===
                class name  precision  recall  F1-score  support
0                     food      0.622   0.831     0.711      261
1                    price      0.500   0.099     0.165       71
2                  service      0.600   0.425     0.498      134
3                 ambience      0.536   0.156     0.242       96
4  anecdotes/miscellaneous      0.640   0.846     0.729      227
5                MACRO AVG      0.579   0.471     0.469      789
6                MICRO AVG      0.621   0.619     0.620      789 

=== CLASSIFICATION : SENTIMENT ===
  class name  precision  recall  F1-score  support
0   positive      0.961   0.941     0.951      440
1   negative      0.831   0.983     0.901      175
2    neutral      0.925   0.860     0.891      100
3   conflict      0.900   0.383     0.537       47
4  MACRO AVG      0.904   0.792     0.820      762
5  MICRO AVG      0.919   0.906     0.912      762 

=== CLASSIFICATION : OVERALL ===
              preci